In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from py_files.c_target_exploration import mod_concat_NEMDE
from py_files.d_concat_DUID_marginal_fuelsource import *

In [2]:
file_path = '{}/nem-data/nemde_clean/concat_for_energy_SA1.pkl'.format(os.environ['HOME'])
df = mod_concat_NEMDE(file_path)

In [3]:
df.head(100)

,RegionID,Market,Price,DUID,DispatchedMarket,Increase
PeriodID,,,,,,
2018-07-31 23:55:00,SA1,Energy,70.70204,ER01,ENOF,0.34712
2018-07-31 23:55:00,SA1,Energy,70.70204,ER04,ENOF,0.31058
2018-07-31 23:55:00,SA1,Energy,70.70204,ER03,ENOF,0.34712
2018-07-31 23:55:00,SA1,Energy,70.70204,ER02,ENOF,0.31058
2018-07-31 23:50:00,SA1,Energy,69.24109,DEVILS_G,L5MI,-1.29132
...,...,...,...,...,...,...
2018-07-31 22:20:00,SA1,Energy,79.30359,ER03,ENOF,0.42336
2018-07-31 22:15:00,SA1,Energy,81.25001,TORRB3,R5MI,0.33333
2018-07-31 22:15:00,SA1,Energy,81.25001,TARONG#1,R5MI,-1.00000


In [5]:
f = df.Price
f.index

DatetimeIndex(['2018-07-31 23:55:00', '2018-07-31 23:55:00',
               '2018-07-31 23:55:00', '2018-07-31 23:55:00',
               '2018-07-31 23:50:00', '2018-07-31 23:50:00',
               '2018-07-31 23:50:00', '2018-07-31 23:50:00',
               '2018-07-31 23:50:00', '2018-07-31 23:50:00',
               ...
               '2009-07-01 04:10:00', '2009-07-01 04:10:00',
               '2009-07-01 04:05:00', '2009-07-01 04:05:00',
               '2009-07-01 04:00:00', '2009-07-01 04:00:00',
               '2009-07-01 04:00:00', '2009-07-01 04:00:00',
               '2009-07-01 04:00:00', '2009-07-01 04:00:00'],
              dtype='datetime64[ns]', name='PeriodID', length=5951707, freq=None)

In [6]:
df[df.DUID == "ENOF,YWPS1,6,YWPS2,5 "]

,RegionID,Market,Price,DUID,DispatchedMarket,Increase
PeriodID,,,,,,


In [7]:
print(df.shape)
df.info()

(5951707, 6)
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5951707 entries, 2018-07-31 23:55:00 to 2009-07-01 04:00:00
Data columns (total 6 columns):
RegionID            object
Market              object
Price               float64
DUID                object
DispatchedMarket    object
Increase            float64
dtypes: float64(2), object(4)
memory usage: 317.9+ MB


In [12]:
file_path = '{}/Desktop/Portfolio_project/DUID_CO2.csv'.format(os.environ['HOME'])
CO2_generators = pd.read_csv(file_path)
print(CO2_generators.DUID.nunique())
CO2_generators.head()

384


,DUID,GENSETID,REGIONID,CO2E_EMISSIONS_FACTOR,CO2E_ENERGY_SOURCE,CO2E_DATA_SOURCE
0,AGLNOW1,AGLNOW1,NSW1,0.062,Landfill biogas methane,NGA 2018
1,AGLSITA1,AGLSITA1,NSW1,0.062,Landfill biogas methane,NGA 2018
2,APPIN,APPIN,NSW1,0.765,Coal seam methane,NGA 2018
3,BDONGHYD,BDONGHYD,NSW1,0.000,Hydro,ISP 2018
4,BERYLSF1,BERYLSF1,NSW1,0.000,Solar,ISP 2018


In [9]:
lst_diff = DUID_items(df, CO2_generators)

The NEMDE_market_table uses 177 DUID items from the DUID CO2_generator column.
The NEMDE_market_table uses 163 DUID items from the GENSETID CO2_generator column.
The items ['ANGAS2', 'ANGAS1', 'MCKAY2'] from the GENSETID column should be added to the DUID column.


In [14]:
all_DUIDs = merge_DUID_GENSETID(CO2_generators, lst_diff)
print(all_DUIDs.DUID.nunique())
all_DUIDs.head()

387


,DUID,CO2E_EMISSIONS_FACTOR
0,AGLHAL,1.174440
1,AGLNOW1,0.062000
2,AGLSITA1,0.062000
3,AGLSOM,0.817915
4,ANGAS1,1.013600


In [15]:
file_path = '{}/nem-data/nemde_clean/all_DUIDs.pkl'.format(os.environ['HOME'])
pd.to_pickle(all_DUIDs, file_path)

In [17]:
df_DUID_CO2 = df.reset_index().merge(all_DUIDs, how="left", on="DUID").set_index("PeriodID")
df_DUID_CO2 = df_DUID_CO2.drop(columns=["RegionID", "Market", "DispatchedMarket", "Increase"])
print(df_DUID_CO2.shape)
df_DUID_CO2

(5951707, 3)


,Price,DUID,CO2E_EMISSIONS_FACTOR
PeriodID,,,
2018-07-31 23:55:00,70.70204,ER01,0.910143
2018-07-31 23:55:00,70.70204,ER04,0.910143
2018-07-31 23:55:00,70.70204,ER03,0.910143
2018-07-31 23:55:00,70.70204,ER02,0.910143
2018-07-31 23:50:00,69.24109,DEVILS_G,0.000000
...,...,...,...
2009-07-01 04:00:00,1.35918,ER03,0.910143
2009-07-01 04:00:00,1.35918,ER04,0.910143
2009-07-01 04:00:00,1.35918,"ENOF,YWPS2,2,YWPS3,2",NaN


In [19]:
print(df_DUID_CO2[~df_DUID_CO2.CO2E_EMISSIONS_FACTOR.isna()].shape)
df_DUID_CO2 = df_DUID_CO2[~df_DUID_CO2.CO2E_EMISSIONS_FACTOR.isna()]
df_DUID_CO2

(2886240, 3)


,Price,DUID,CO2E_EMISSIONS_FACTOR
PeriodID,,,
2018-07-31 23:55:00,70.70204,ER01,0.910143
2018-07-31 23:55:00,70.70204,ER04,0.910143
2018-07-31 23:55:00,70.70204,ER03,0.910143
2018-07-31 23:55:00,70.70204,ER02,0.910143
2018-07-31 23:50:00,69.24109,DEVILS_G,0.000000
...,...,...,...
2009-07-01 04:00:00,1.35918,ER02,0.910143
2009-07-01 04:00:00,1.35918,ER03,0.910143
2009-07-01 04:00:00,1.35918,ER04,0.910143


In [36]:
df_DUID_CO2_CO2 = df_DUID_CO2.groupby(df_DUID_CO2.index).mean().sort_index(ascending=False)
print(df_DUID_CO2_CO2.shape)
df_DUID_CO2_CO2.head()

(911397, 2)


,Price,CO2E_EMISSIONS_FACTOR
PeriodID,,
2018-07-31 23:55:00,70.70204,0.910143
2018-07-31 23:50:00,69.24109,0.728114
2018-07-31 23:45:00,69.94850,0.910143
2018-07-31 23:40:00,74.04497,0.908305
2018-07-31 23:35:00,76.44245,0.971761


In [37]:
print(df_DUID_CO2_CO2.Price.max())
print(df_DUID_CO2_CO2.Price.min())
print(sum(df_DUID_CO2_CO2.Price > 13500))
print(sum(df_DUID_CO2_CO2.Price < -1000))

1202903.49417
-26801.01056999999
156
65


In [38]:
df_DUID_CO2_price = df_DUID_CO2_time.query('Price > -1000 and Price < 13500')
print(sum(df_DUID_CO2_price.Price > 13500))
print(sum(df_DUID_CO2_price.Price < -1000))

0
0


In [40]:
len(pd.date_range(start='2009-07-01 04:00:00', end='2018-07-31 23:55:00', freq='5min'))

955536

In [41]:
def correct_timedelta(df, time_diff):
    '''
    df.index must be DateTimeIndex
    table=table_of_interest
    col="column_of_interest"
    time_diff=time_diff in seconds as int
    '''
    lst = []
    lst_i = []
    count = 0
    for i in df.index:
        count += 1
        if count >= len(df):
            break
        delta = abs(df.index[count] - df.index[count-1])
        if int(delta.total_seconds()) != int(time_diff):
            lst.append(("from index {} on, it has been {} s or {} h.".format(count-1,int(delta.total_seconds()),(int(delta.total_seconds()/3600)))))
            lst_i.append((count-1,int(delta.total_seconds())))
    return lst, lst_i

In [45]:
a1,b1 = correct_timedelta(df_DUID_CO2_CO2, 300)

In [46]:
len(a1)

24983

In [47]:
a2,b2 = correct_timedelta(df_DUID_CO2_price, 300)

In [48]:
len(a2)

25131

In [50]:
print(df_DUID_CO2_price.CO2E_EMISSIONS_FACTOR.min())
print(df_DUID_CO2_price.CO2E_EMISSIONS_FACTOR.max())

0.0
1.4918


In [53]:
bins = list(np.arange(df_DUID_CO2_price.CO2E_EMISSIONS_FACTOR.min(),
                      df_DUID_CO2_price.CO2E_EMISSIONS_FACTOR.max(), 
                      df_DUID_CO2_price.CO2E_EMISSIONS_FACTOR.max()/10))
type(bins)
group_names = bins[:-1]
group_names

[0.0,
 0.14918,
 0.29836,
 0.44754000000000005,
 0.59672,
 0.7459,
 0.8950800000000001,
 1.04426,
 1.19344]

In [57]:
df_DUID_CO2_price["CO2_group"] = pd.cut(df_DUID_CO2_price.CO2E_EMISSIONS_FACTOR, bins, labels=group_names)

In [58]:
df_DUID_CO2_price.head()

,Price,CO2E_EMISSIONS_FACTOR,CO2_group
PeriodID,,,
2018-07-31 23:55:00,70.70204,0.910143,0.89508
2018-07-31 23:50:00,69.24109,0.728114,0.59672
2018-07-31 23:45:00,69.94850,0.910143,0.89508
2018-07-31 23:40:00,74.04497,0.908305,0.89508
2018-07-31 23:35:00,76.44245,0.971761,0.89508


In [61]:
df_DUID_CO2_price.groupby("CO2_group").mean()

,Price,CO2E_EMISSIONS_FACTOR
CO2_group,,
0.00000,-113.974696,0.117165
0.14918,39.850092,0.232275
0.29836,53.177382,0.381765
0.44754,58.867481,0.508666
0.59672,85.598736,0.683207
0.74590,66.327211,0.835431
0.89508,41.213792,0.937934
1.04426,44.395009,1.122903
1.19344,224.947067,1.308523


In [ ]:
file_path = '{}/nem-data/nemde_clean/df_DUID_CO2_price.pkl'.format(os.environ['HOME'])
df_DUID_CO2_price = pd.to_pickle(df_DUID_CO2_price, file_path)